In [25]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.options.display.max_columns = None

In [2]:
df = pd.read_csv("attacks_columnas_limpias copy.csv", index_col = 0)
df.head(2)

,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,href,fatal_(y/n),injury,date,age_new
0,1800.00.00,1927.133059,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,F,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,"FATAL, all onboard were killed by sharks",1800,NaN
1,1797.05.28.R,1927.133059,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,Y,FATAL,Reported May-28-1797,NaN


### EJERCICIO 1:
Es el momento de eliminar los nulos:
 - Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [17]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

In [20]:
imputer = imputer.fit(df[['age_new']])

In [21]:
df['age_new'] = imputer.transform(df[['age_new']])

 - Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 ** Pista** 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [3]:
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)

In [4]:
imputer = imputer.fit(df[['sex_']])

In [5]:
df['sex_mode'] = imputer.transform(df[['sex_']])

In [24]:
df.isnull().sum()

case_number       1
year              0
type              4
country          50
area            456
location        542
activity        544
name            209
sex_            569
age            2820
time           3364
species_       2845
href              0
fatal_(y/n)     546
injury           28
date              0
age_new           0
sex_mode          0
type_mode         0
dtype: int64

In [7]:
df['sex_mode'].unique()

array(['F', 'M', 'M ', 'lli', 'N', '.'], dtype=object)

<font color ='orange'>Hemos creado una columna nueva para sex, rellenando los valores nulos por aquellos valores más frecuentes.

 - Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [8]:
imputer = imputer.fit(df[['type']])

In [9]:
df['type_mode'] = imputer.transform(df[['type']])

In [23]:
df.isnull().sum()

case_number       1
year              0
type              4
country          50
area            456
location        542
activity        544
name            209
sex_            569
age            2820
time           3364
species_       2845
href              0
fatal_(y/n)     546
injury           28
date              0
age_new           0
sex_mode          0
type_mode         0
dtype: int64

In [11]:
df['type_mode'].unique()

array(['Unprovoked', 'Provoked', 'Boat', 'Sea Disaster', 'Invalid',
       'Boating', 'Questionable', 'Boatomg'], dtype=object)

<font color ='orange'>Mantenemos el imputer del ejercicio anterior ya que continuamos obteniendo la moda de los valores. Comprobamos que ya no tenemos nulos en la nueva columna.

 - Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [27]:
numericas = df.select_dtypes(include = np.number)

In [28]:
numericas.isnull().sum()

year       0
age_new    0
dtype: int64

In [14]:
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(numericas)

KNNImputer()

In [15]:
numericas_knn= imputerKNN.transform(numericas)

In [16]:
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)

In [38]:
columnas_knn = df_knn_imputer.columns

In [39]:
df.drop(columnas_knn, axis = 1, inplace = True)

In [40]:
df[columnas_knn] = numericas_knn

In [41]:
df.isnull().sum()

case_number       1
type              4
country          50
area            456
location        542
activity        544
name            209
sex_            569
age            2820
time           3364
species_       2845
href              0
fatal_(y/n)     546
injury           28
date              0
sex_mode          0
type_mode         0
year              0
age_new           0
dtype: int64

 - Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

<font color ='orange'> Trabajamos con las columnas age y year al ser las únicas numéricas.

In [26]:
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')

In [29]:
imputer.fit(numericas)

IterativeImputer()

In [30]:
imputer.transform(numericas)

array([[1927.13305942,   27.26768258],
       [1927.13305942,   27.26768258],
       [1927.13305942,   27.26768258],
       ...,
       [1927.13305942,   27.26768258],
       [1927.13305942,   27.26768258],
       [1927.13305942,   27.26768258]])

In [31]:
numericas_trans = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns)

In [32]:
numericas_trans.head()

,year,age_new
0,1927.133059,27.267683
1,1927.133059,27.267683
2,1927.133059,27.267683
3,1927.133059,27.267683
4,1927.133059,27.267683


In [42]:
columnas = numericas_trans.columns

In [43]:
df.drop(columnas, axis = 1, inplace = True)

In [44]:
df[columnas] = numericas_trans[columnas]

In [45]:
df.isnull().sum()

case_number       1
type              4
country          50
area            456
location        542
activity        544
name            209
sex_            569
age            2820
time           3364
species_       2845
href              0
fatal_(y/n)     546
injury           28
date              0
sex_mode          0
type_mode         0
year              0
age_new           0
dtype: int64

<font color = 'orange'>La diferencia entre el método KNN Imputer y Iterative Imputer es que con el segundo tenemos que crear unas columnas nuevas donde cambiaremos los datos nulos por otro valor estadístico del conjunto, con el objetivo de eliminar las que teníamos en primer lugar y sustituirlas por las nuevas. En cambio, el método KNN Imputer se basa en predecir el valor numérico nulo promediando las distancias entre sus k vecinos más cercanos.

EJERCICIO 2: Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair. 

In [48]:
df.to_csv('attacks_limpieza_5.csv')